In [ ]:
!gcloud auth activate-service-account --key-file=untitled.txt --project=spark-on-kubernetes-testing

In [ ]:
!gcloud container clusters get-credentials spark-cluster --region=us-central1

In [ ]:
!kubectl cluster-info

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType, ArrayType, LongType
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
spark = SparkSession \
    .builder \
    .appName("kubernetes") \
    .getOrCreate()

In [ ]:
spark.stop()

In [ ]:
# Create Spark Session and Load BigQuery jar file
spark = SparkSession \
    .builder \
    .appName("kubernetes") \
    .config("spark.master","k8s://https://35.239.173.65:443") \
    .config("spark.submit.deployMode","cluster") \
    .config("spark.executor.instances","3") \
    .config("spark.kubernetes.container.image","gcr.io/spark-on-kubernetes-testing/spark-cluster-spark-base:latest") \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName","spark") \
    .getOrCreate()

In [ ]:
# Define schema of files to parse
schema = StructType([ 
    StructField("asin",StringType(),True), 
    StructField("image",ArrayType(StringType()),True), 
    StructField("overall",DoubleType(),True),
    StructField("reviewText",StringType(),True),
    StructField("reviewTime",StringType(),True),
    StructField("reviewerID",StringType(),True),
    StructField("reviewerName",StringType(),True),
    StructField("summary",StringType(),True),
    StructField("unixReviewTime",LongType(),True),
    StructField("verified",BooleanType(),True),
    StructField("vote",StringType(),True),
    StructField("style",ArrayType(StringType()),True)
  ])

# URL to scrape to get files to download
url = "https://nijianmo.github.io/amazon/index.html"
html = requests.get(url)

if html.ok:
    soup = BeautifulSoup(html.content, 'html.parser')  

output_final = []
files = []
links = soup.find_all('a',string='5-core')#.find('5-core')#.find_all('td', id='5-core')
for link in links[0:1]:
    url = link.get('href')
    file = url.split('/')[-1]
    print(url)
    print(url.split('/')[-1])
    spark.sparkContext.addFile(url)
    files.append(file)
    df = spark.read.json("file://"+SparkFiles.get(file),schema)

df.show(5)